In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import resnet18, ResNet18_Weights
import matplotlib.pyplot as plt
import time
import copy
import os
import cv2
import torch
from PIL import Image
import torchvision.transforms as transforms # not to be confused with tensorflow
from torchvision.transforms.functional import crop
from io import BytesIO
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split

from google.colab import drive
drive.mount('/content/drive')

torch.manual_seed(42)
torch.cuda.manual_seed(42)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/Machine Learning/COSMOS/Final Project/Emotion Recognition/radset.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = torch.load(path)
train_length = int(0.9 * len(data))
train_data, test_data = torch.utils.data.random_split(data, [train_length, len(data) - train_length])
print(train_length, len(data) - train_length)

1296 144


In [ ]:
# using resnet50 from the pytorch library
#model.fc.register_forward_hook(lambda m, inp, out: F.dropout(out, p=0.5, training=m.training))

val_acc = []
val_loss = []

trainbsize = 8
testbsize = 8

training_acc = []
trainin_loss = []
train_loader = DataLoader(train_data, batch_size = trainbsize, shuffle = True, pin_memory = True, num_workers = 2)
test_loader = DataLoader(test_data, batch_size = testbsize, shuffle = False, pin_memory = True, num_workers = 2)

def preprocess_again(image, label, batch):
  label = label.squeeze(1)
  return image, label

In [ ]:
epochs = 15
model = resnet18(pretrained = True).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 1e-5, nesterov = True)

val_acc = []
val_loss = []
training_acc = []
training_loss = []

for epoch in range(1, epochs+1):
  model.train()
  train_loss = 0.00
  correct = 0.00
  total = 0.00
  for image, label in train_loader:
    image, label = preprocess_again(image, label, trainbsize)
    image = image.cuda()
    label = label.cuda()
    optimizer.zero_grad()
    y_predict = model(image).to(device)
    loss = criterion(y_predict, label)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
    _, prediction = y_predict.max(dim=1)
    correct += (prediction.eq(label).sum().item())
    total += label.size(0)
  training_acc.append(correct / total)
  training_loss.append(train_loss/len(train_loader))
  print(f"Epoch: [{epoch}/{epochs}], Training Accuracy: {training_acc[epoch - 1]:.4f}, Training Loss: {training_loss[epoch - 1]:.4f}")

Epoch: [1/15], Training Accuracy: 0.4452, Training Loss: 2.0210
Epoch: [2/15], Training Accuracy: 0.7361, Training Loss: 0.7479
Epoch: [3/15], Training Accuracy: 0.8688, Training Loss: 0.3836
Epoch: [4/15], Training Accuracy: 0.9213, Training Loss: 0.2363
Epoch: [5/15], Training Accuracy: 0.9622, Training Loss: 0.1313
Epoch: [6/15], Training Accuracy: 0.9722, Training Loss: 0.0865
Epoch: [7/15], Training Accuracy: 0.9807, Training Loss: 0.0717
Epoch: [8/15], Training Accuracy: 0.9769, Training Loss: 0.0692
Epoch: [9/15], Training Accuracy: 0.9830, Training Loss: 0.0606
Epoch: [10/15], Training Accuracy: 0.9730, Training Loss: 0.0849
Epoch: [11/15], Training Accuracy: 0.9853, Training Loss: 0.0529
Epoch: [12/15], Training Accuracy: 0.9892, Training Loss: 0.0362
Epoch: [13/15], Training Accuracy: 0.9892, Training Loss: 0.0354
Epoch: [14/15], Training Accuracy: 0.9938, Training Loss: 0.0310
Epoch: [15/15], Training Accuracy: 0.9969, Training Loss: 0.0149


In [ ]:
model.eval()
correct = 0.00
total = 0.00
_val_loss = 0.00

with torch.no_grad():
  for image, label in test_loader:
    image, label = preprocess_again(image, label, testbsize)
    image = image.cuda()
    label = label.cuda()
    y_pred = model(image).to(device)
    _, prediction = y_pred.max(dim=1)
    correct += prediction.eq(label).sum().item()
    total += label.size(0)

print(f"Validation Accuracy: {correct/total:4f}")

Validation Accuracy: 0.798611
